In [25]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats
import matplotlib.pyplot as plt
from tqdm import tqdm

# Regressão Linear

Vamos iniciar ajustando nossos dados normalmente com o auxílio da biblioteca statsmodels

Lendo os dados

In [26]:
df_reservatorio = pd.read_csv('clorofila.csv')

In [27]:
df_reservatorio.head()

,Condutividade,DO,pH,ORP,Profundidade,Temperatura,Turbidez,Clorofila
0,121.73,8.32,7.47,198.04,0.37,20.55,6.98,59.97
1,121.29,5.67,6.90,231.30,0.40,19.98,4.65,30.59
2,120.77,5.33,6.79,229.20,0.42,19.61,3.94,17.06
3,119.50,7.61,7.10,228.61,0.41,19.47,5.20,38.00
4,121.17,7.96,7.12,208.77,0.39,19.73,4.76,24.39


X será uma series contendo a variável preditora, pH

Y será uma series com os valores de Clorofila, nossa variável resposta

In [28]:
X = df_reservatorio['pH']
Y = df_reservatorio['Clorofila'] 

Aqui, precisamos garantir que exista intercepto

In [29]:
X = sm.add_constant(X)

Inicializando o modelo e startando o processo de busca dos parâmetros Betas

In [30]:
modelo_linear_mult = sm.OLS(Y, X)
modelo_linear_mult = modelo_linear_mult.fit()

Sumarizando os resultados na tabela

In [31]:
print(modelo_linear_mult.summary())

                            OLS Regression Results                            
Dep. Variable:              Clorofila   R-squared:                       0.283
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     200.9
Date:                Thu, 29 Dec 2022   Prob (F-statistic):           1.08e-38
Time:                        23:21:33   Log-Likelihood:                -2013.2
No. Observations:                 512   AIC:                             4030.
Df Residuals:                     510   BIC:                             4039.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -52.6095      5.710     -9.214      0.0

# Gradiente Descendente

Vamos definir inicialmente listas para guardar os valores de MSE, Beta 0 e Beta 1 a cada iteração que será feita.

In [32]:
np.random.seed(100)
MSE = []
B1 = []
B0 = []
B1.append(np.random.rand())
B0.append(np.random.rand())


Vamos agora definir os valores de alfa (taxa de aprendizado) e número de iterações n_iter


In [33]:
X_ph = df_reservatorio['pH']
alfa = 0.01
n_inter = 20000

Vamos criar funções para:
* Calcular o MSE
* Calcular o Gradiente em relação a B0
* Calcular o Gradiente em relação a B1

In [34]:
# MSE
def mean_squared_error(y_real , y_previsto):
    return np.mean((y_real - y_previsto)**2)


In [35]:
#Gradiente BO
def gradiente_b0(Y, y_previsto):
    gradient = (-2/len(y_previsto))*np.sum(Y-y_previsto)
    return gradient

In [36]:
#Gradiente B1
def gradiente_b1(Y, y_previsto, X):
    gradient = ((-2/len(y_previsto))*(np.sum((Y-y_previsto)*X)))
    return gradient

In [37]:
for i in tqdm(range(n_inter)):
    y_previsto = B1[i]*X_ph + B0[i]
    B0.append(B0[i] - alfa*gradiente_b0(Y, y_previsto))
    B1.append(B1[i] - alfa*gradiente_b1(Y, y_previsto , X_ph))
    MSE.append(mean_squared_error(Y, y_previsto))


100%|██████████| 20000/20000 [00:25<00:00, 789.72it/s] 


In [41]:
plt.plot(range(n_inter) , MSE)
plt.xlabel('interação')
plt.ylabel('Clorofila')

for i in tqdm(range(n_inter+1)):
    y_previsto_gd = B0[i] + B1[i]*X_ph
    plt.plot(X_ph , y_previsto_gd , color = 'red', alpha = 0.05)
    if i == n_inter:
        plt.plot(X_ph, y_previsto_gd, color = 'black')
plt.show()

In [39]:
B0[-1]

-51.160472460425964

In [40]:
B1[-1]

10.30893030095601